## MLflow's Model Registry

In [46]:
from mlflow.tracking import MlflowClient
from pprint import pprint

# The project we already created URL
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [47]:
# Create a client
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# client.list_experiments()
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1686585488370,
    'description': 'The NYC Taxi prediction',
    'last_updated_timestamp': 1686615459235,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1686595083171, current_stage='None', description='', last_updated_timestamp=1686595083171, name='nyc_taxi_regressor', run_id='0827e808120245f58fcfe94f6a4621dd', run_link='', source='/Users/amberm/PycharmProjects/MLops/MLops/02-experiment-tracking/mlruns/3/0827e808120245f58fcfe94f6a4621dd/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>,
                           <ModelVersion: aliases=[], creation_timestamp=1686595095992, current_stage='Staging', description='', last_updated_timestamp=1686615444734, name='nyc_taxi_regressor', run_id='2f1bb039ed0f4c46aa4045e45146f6e8', run_link='', source='/Users/amberm/PycharmProjects/MLops/MLops/02-experiment-tracking/mlruns/4/2f1bb039ed0f4c46aa4045e45146f6e8/artifacts/model', status

In [48]:
client.create_experiment(name="new_Selection")
# The 6th one in mlflow

'7'

In [49]:
# Check the latest versions for the experiment with id `1`...

from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='4',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [50]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 1cf87a8d01a24711b43a341bf65c8043, rmse: 6.7423
run id: 218a991b5da348108b9b09201bc9c5de, rmse: 6.9036
run id: ce5531fed3194be0be10b40f2bb2fb96, rmse: 6.9124
run id: be9838855685495d895850f3ffd19c0a, rmse: 6.9151


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [51]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [52]:
# Add the best model to registry
run_id = "1cf87a8d01a24711b43a341bf65c8043"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc_taxi_regressor")

Registered model 'nyc_taxi_regressor' already exists. Creating a new version of this model...
2023/06/12 21:23:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc_taxi_regressor, version 7
Created version '7' of model 'nyc_taxi_regressor'.


<ModelVersion: aliases=[], creation_timestamp=1686619423188, current_stage='None', description=None, last_updated_timestamp=1686619423188, name='nyc_taxi_regressor', run_id='1cf87a8d01a24711b43a341bf65c8043', run_link=None, source='/Users/amberm/PycharmProjects/MLops/MLops/02-experiment-tracking/mlruns/4/1cf87a8d01a24711b43a341bf65c8043/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>

In [53]:
# Get the list of registry
model_name = "nyc_taxi_regressor"
# change from list_registered_model() to get_registered_model()
client.get_registered_model(name="nyc_taxi_regressor")

<RegisteredModel: aliases={}, creation_timestamp=1686585488370, description='The NYC Taxi prediction', last_updated_timestamp=1686619423188, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1686619423188, current_stage='None', description=None, last_updated_timestamp=1686619423188, name='nyc_taxi_regressor', run_id='1cf87a8d01a24711b43a341bf65c8043', run_link=None, source='/Users/amberm/PycharmProjects/MLops/MLops/02-experiment-tracking/mlruns/4/1cf87a8d01a24711b43a341bf65c8043/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>,
 <ModelVersion: aliases=[], creation_timestamp=1686595095992, current_stage='Staging', description='', last_updated_timestamp=1686615444734, name='nyc_taxi_regressor', run_id='2f1bb039ed0f4c46aa4045e45146f6e8', run_link='', source='/Users/amberm/PycharmProjects/MLops/MLops/02-experiment-tracking/mlruns/4/2f1bb039ed0f4c46aa4045e45146f6e8/artifacts/model', status='READY', status_message=None, tags={'LinearSVR': 

In [54]:
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 7, stage: None
version: 2, stage: Staging
version: 6, stage: Archived
version: 4, stage: Production


In [55]:
# Changing the current version stage
model_version = 6
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1686595700621, current_stage='Staging', description='The model version 6 was transitioned to Staging on 2023-06-12', last_updated_timestamp=1686619423214, name='nyc_taxi_regressor', run_id='1cf87a8d01a24711b43a341bf65c8043', run_link=None, source='/Users/amberm/PycharmProjects/MLops/MLops/02-experiment-tracking/mlruns/4/1cf87a8d01a24711b43a341bf65c8043/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [56]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1686595700621, current_stage='Staging', description='The model version 6 was transitioned to Staging on 2023-06-12', last_updated_timestamp=1686619423224, name='nyc_taxi_regressor', run_id='1cf87a8d01a24711b43a341bf65c8043', run_link=None, source='/Users/amberm/PycharmProjects/MLops/MLops/02-experiment-tracking/mlruns/4/1cf87a8d01a24711b43a341bf65c8043/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [57]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [58]:
df = read_dataframe('./data/green_tripdata_2021-03.parquet')

In [59]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/var/folders/0m/hd5nyhdj19b3dcxl72fwby3c0000gn/T/ipykernel_36250/2018306454.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/Users/amberm/PycharmProjects/MLops/MLops/02-experiment-tracking/preprocessor'

In [60]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [61]:
X_test = preprocess(df, dv)

In [62]:
target = "duration"
y_test = df[target].values

In [63]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 65.2 ms, sys: 7.01 ms, total: 72.2 ms
Wall time: 86.5 ms


{'rmse': 6.659623830022514}

In [64]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 63.7 ms, sys: 1.92 ms, total: 65.6 ms
Wall time: 64.8 ms


{'rmse': 6.659623830022514}

In [65]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1686595110491, current_stage='Production', description='', last_updated_timestamp=1686619423971, name='nyc_taxi_regressor', run_id='1cf87a8d01a24711b43a341bf65c8043', run_link='', source='/Users/amberm/PycharmProjects/MLops/MLops/02-experiment-tracking/mlruns/4/1cf87a8d01a24711b43a341bf65c8043/artifacts/model', status='READY', status_message=None, tags={'GradientBoostingRegressor': 'GradientBoostingRegressor'}, user_id=None, version=4>